In [ ]:
import nltk

In [ ]:
datafile = ''
embeddingfile = ''

In [ ]:
seed = 42


In [ ]:
vocab_size = 8000

In [ ]:
embedding_dim = 50

In [ ]:
lower = False

In [ ]:
with open():
     data = ''
    

In [ ]:
def load_embeddings():
  with open('../word_embedding/glove.6B/glove.6B.50d.txt') as f:
    words = {}
    w = [line.split(' ') for line in f]
    print(w[0])
    v = [x[1:] for x in w]
    w = [x[0] for x in w]
    print(len(v))
    print(len(w))

    for i in range(len(w)):
      words[w[i]] = v[i]

    return(words)
